In [1]:
import selenium
from selenium import webdriver
import time
from time import sleep
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.webdriver.common.by import By
import numpy as np
import csv
import pandas as pd

pd.set_option('display.max_columns', None)

In [2]:
def get_start_url(data_page, links_list):
    # generate the urls for each restaurant
    links = []
    try:
        if data_page.find_elements('xpath','//div[@data-testid="serp-ia-card"]//div[@aria-label="Slideshow"]//a'):
            restaurants = data_page.find_elements('xpath','//div[@data-testid="serp-ia-card"]//div[@aria-label="Slideshow"]//a')
            for r in restaurants:
                link = r.get_attribute('href')
                # ignore the previous crawled ones
                if link not in links_list:
                    links.append(link)
            
    except NoSuchElementException:
        links.append('')
    return links
    

In [3]:
def get_details(url):
    # crawl the detailed information of each restaurant
    driver = webdriver.Chrome(executable_path="C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe",options=options)
    driver.set_window_size(1920,1080)
    driver.get(url)
    wait = WebDriverWait(driver, 1)
    
    try:
        if driver.find_element('xpath','//h1[@class="css-1se8maq"]'):
            name = driver.find_element('xpath','//h1[@class="css-1se8maq"]').text
    except NoSuchElementException:
        name = ''
        
    try:
        if driver.find_element('xpath','.//address'):
            address = ' '.join(driver.find_element('xpath','.//address').text.split('\n'))
    except NoSuchElementException:
        address = ''
        
    try:
        if driver.find_element('xpath','//div[@data-testid="photoHeader"]//div[@class=" five-stars__09f24__mBKym five-stars--large__09f24__Waiqf display--inline-block__09f24__fEDiJ  border-color--default__09f24__NPAKY"][@aria-label]'):
            score = driver.find_element('xpath','//div[@data-testid="photoHeader"]//div[@class=" five-stars__09f24__mBKym five-stars--large__09f24__Waiqf display--inline-block__09f24__fEDiJ  border-color--default__09f24__NPAKY"][@aria-label]').get_attribute('aria-label')
    except NoSuchElementException:
        score = ''
        
    try:
        if driver.find_element('xpath','//a[@href="#reviews"]'):
            review_counts = driver.find_element('xpath','//a[@href="#reviews"]').text
    except NoSuchElementException:
        review_counts = ''
        
    try:
        if driver.find_element('xpath','//span[@class=" css-1ir4e44"]'):
            price = driver.find_element('xpath','//span[@class=" css-1ir4e44"]').text
    except NoSuchElementException:
        price = ''
        
    try:
        if driver.find_element('xpath','//div[@data-testid="photoHeader"]//span[@class=" display--inline__09f24__c6N_k margin-r1-5__09f24__ot4bd  border-color--default__09f24__NPAKY"]'):
             category = driver.find_element('xpath','//div[@data-testid="photoHeader"]//span[@class=" display--inline__09f24__c6N_k margin-r1-5__09f24__ot4bd  border-color--default__09f24__NPAKY"]').text
    except NoSuchElementException:
        category = ''
        
    try:
        if driver.find_element('xpath','//table[@class="hours-table__09f24__KR8wh css-1jtriov"]'):
            open_time = driver.find_element('xpath','//table[@class="hours-table__09f24__KR8wh css-1jtriov"]').text
    except NoSuchElementException:
        open_time = ''
    
    try:
        if driver.find_element('xpath','.//section[@aria-label="Amenities and More"]//button[@type="submit"]//p'):
            try:
                driver.find_element('xpath','.//section[@aria-label="Amenities and More"]//button[@type="submit"]//p').click()
            except ElementNotInteractableException:
                button = False
        if driver.find_element('xpath','//section[@aria-label="Amenities and More"]'):
            amenities = driver.find_element('xpath','//section[@aria-label="Amenities and More"]').text
    except NoSuchElementException:
        amenities = ''
    
    driver.quit()
    return [url, name,address,score,review_counts,price,category,open_time,amenities]
    

In [39]:
def generate_base_urls(base_url_list):
    """generate the urls based on the first page of each category"""
    urls = []
    page_num = np.arange(0,240,10)
    for url in base_url_list:
        urls += [url+str(n) for n in page_num]
    return urls

In [5]:
# base_url = 'https://www.yelp.com/search?find_desc=&find_loc=Los+Angeles%2C+CA&start='
page_num = np.arange(0,240,10)
# page_num

In [15]:
base_url_list = [
#     'https://www.yelp.com/search?find_desc=&find_loc=Los+Angeles%2C+CA&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=food&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=mexican&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=breakfast_brunch&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=italian&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=french&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=hawaiian&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=greek&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=cafes&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=chinese&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=barbeque&start='
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=bbq&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=seafood&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=newamerican&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=thai&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=noodles&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=cantonese&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=bars&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=tradamerican&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=latin&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=spanish&start='
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=mediterranean&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=turkish&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=lebanese&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=bakeries&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=argentine&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=soulfood&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=desserts&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=peruvian&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=vegan&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=mideastern&start='
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=african&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=halal&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=bangladeshi&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=indpak&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=buffets&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=foodtrucks&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=juicebars&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=gluten_free&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=icecream&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=gourmet&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=fooddeliveryservices&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=meats&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=grocery&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=markets&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=foodstands&start='
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=wraps&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=acaibowls&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=soup&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=coffee&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=beer_and_wine&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=tapasmallplates&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=poke&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=catering&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=personalchefs&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=cajun&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=chicken_wings&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=korean&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=waffles&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=newmexican&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=hotdog&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=pastashops&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=bubbletea&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=taiwanese&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=creperies&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=kosher&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=donuts&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=eventservices&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=popuprestaurants&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=kebab&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los+Angeles%2C+CA&cflt=hotels&start='
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Alhambra%2C+CA&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Arcadia%2C+CA&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Beverly+Hills%2C+CA&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Burbank%2C+CA&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Claremont%2C+CA&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Culver+City%2C+CA&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=El+Monte%2C+CA&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Glendale%2C+CA&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Inglewood%2C+CA&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Compton%2C+CA&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=West+Hollywood%2C+CA&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Venice%2C+Los+Angeles%2C+CA&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Northridge%2C+Los+Angeles%2C+CA&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=North+Hollywood%2C+Los+Angeles%2C+CA&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=West+Hills%2C+Los+Angeles%2C+CA&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Playa+Vista%2C+Los+Angeles%2C+CA&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Pacific+Palisades%2C+Los+Angeles%2C+CA&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Porter+Ranch%2C+Los+Angeles%2C+CA&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Bell+Gardens%2C+CA&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Marina+del+Rey%2C+CA&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=San+Fernando%2C+CA&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Paramount%2C+CA&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Woodland+Hills%2C+Los+Angeles%2C+CA&start=',
#     'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Pasadena%2C+CA&start=',
    'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Pasadena%2C+CA&start='
]

In [16]:
# generate the base url for each page
urls = generate_base_urls(base_url_list)

In [17]:
urls[:2]  # each yelp search result has 24 pages

['https://www.yelp.com/search?find_desc=Restaurants&find_loc=Alhambra%2C+CA&start=0',
 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Alhambra%2C+CA&start=10']

In [18]:
restaurant_links = []

In [42]:
options = webdriver.ChromeOptions()
options.add_argument("--headless")

In [8]:
new_url_df  = pd.read_csv('restaurant_all_test.csv')
new_urls= new_url_df['url'].tolist()

In [12]:
len(list(set(new_urls)))

8667

In [19]:
# load the current crawled links
links = pd.read_csv('current_links.csv')
links_list = links['url'].tolist()

In [20]:
print(f"current number of links={len(links_list)}")

current number of links=9865


In [29]:
links_list[:5]

['https://www.yelp.com/biz/plan-check-kitchen-bar-los-angeles-9',
 'https://www.yelp.com/biz/justin-quesos-tex-mex-restaurant-and-bar-west-hollywood?osq=Restaurants',
 'https://www.yelp.com/biz/angelini-alimentari-los-angeles?osq=Restaurants',
 'https://www.yelp.com/biz/c-and-o-trattoria-marina-del-rey-6?osq=Restaurants',
 'https://www.yelp.com/biz/sunny-grill-los-angeles?osq=Restaurants']

In [30]:
tmp_list = []
for url in links_list:
    url = url.split('?')[0]
    tmp_list.append(url)

In [31]:
tmp_list[:5]

['https://www.yelp.com/biz/plan-check-kitchen-bar-los-angeles-9',
 'https://www.yelp.com/biz/justin-quesos-tex-mex-restaurant-and-bar-west-hollywood',
 'https://www.yelp.com/biz/angelini-alimentari-los-angeles',
 'https://www.yelp.com/biz/c-and-o-trattoria-marina-del-rey-6',
 'https://www.yelp.com/biz/sunny-grill-los-angeles']

In [45]:
new_url_to_crawl = []
for new_url in new_urls:
    new_url=new_url.split('?')[0]
    if new_url not in tmp_list:
        new_url_to_crawl.append(new_url)

In [33]:
print(f"number of new links = {len(new_url_to_crawl)}")

number of new links = 5381


In [34]:
new_url_to_crawl[:5]

['https://www.yelp.com/biz/granville-west-hollywood-2',
 'https://www.yelp.com/biz/great-white-los-angeles-2',
 'https://www.yelp.com/biz/met-him-at-a-bar-los-angeles-3',
 'https://www.yelp.com/biz/kateen-los-angeles-2',
 'https://www.yelp.com/biz/king-chang-la-los-angeles']

In [22]:
# crawl the urls for each restaurant
for url in urls:
    # Provide the path of chromedriver present on your system.
    driver = webdriver.Chrome(executable_path="C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe",options=options)
    driver.set_window_size(1920,1080)
    # Send a get request to the url
    driver.get(url)
    wait = WebDriverWait(driver, 3)
    data_page = wait.until(
    EC.presence_of_element_located(
    (By.XPATH, """//*[@id="main-content"]""")))
    # don't crawl the previously downloaded one
    links = get_start_url(driver, links_list)
    restaurant_links += links
    print(f"total urls: {len(restaurant_links)},unique urls: {len(list(set(restaurant_links)))}")
    driver.quit()

total urls: 75,unique urls: 65
total urls: 84,unique urls: 65
total urls: 94,unique urls: 65
total urls: 103,unique urls: 65
total urls: 111,unique urls: 65
total urls: 121,unique urls: 65
total urls: 130,unique urls: 65
total urls: 137,unique urls: 72
total urls: 147,unique urls: 82
total urls: 154,unique urls: 89
total urls: 164,unique urls: 99
total urls: 173,unique urls: 107
total urls: 183,unique urls: 117
total urls: 192,unique urls: 126
total urls: 202,unique urls: 136
total urls: 211,unique urls: 145
total urls: 221,unique urls: 155
total urls: 230,unique urls: 164
total urls: 240,unique urls: 174
total urls: 250,unique urls: 184
total urls: 260,unique urls: 194
total urls: 270,unique urls: 204
total urls: 279,unique urls: 213
total urls: 288,unique urls: 222
total urls: 298,unique urls: 228
total urls: 308,unique urls: 236
total urls: 317,unique urls: 244
total urls: 326,unique urls: 245
total urls: 336,unique urls: 252
total urls: 345,unique urls: 259
total urls: 354,unique u

total urls: 1663,unique urls: 1317
total urls: 1664,unique urls: 1318
total urls: 1665,unique urls: 1318
total urls: 1668,unique urls: 1320
total urls: 1669,unique urls: 1320
total urls: 1670,unique urls: 1321
total urls: 1672,unique urls: 1322
total urls: 1673,unique urls: 1322
total urls: 1674,unique urls: 1322
total urls: 1676,unique urls: 1324
total urls: 1677,unique urls: 1325
total urls: 1678,unique urls: 1325
total urls: 1679,unique urls: 1326
total urls: 1681,unique urls: 1327
total urls: 1682,unique urls: 1327
total urls: 1684,unique urls: 1328
total urls: 1687,unique urls: 1330
total urls: 1688,unique urls: 1331
total urls: 1691,unique urls: 1333
total urls: 1693,unique urls: 1335
total urls: 1693,unique urls: 1335
total urls: 1693,unique urls: 1335
total urls: 1693,unique urls: 1335
total urls: 1693,unique urls: 1335
total urls: 1695,unique urls: 1336
total urls: 1696,unique urls: 1336
total urls: 1696,unique urls: 1336
total urls: 1697,unique urls: 1337
total urls: 1698,uni

total urls: 2917,unique urls: 2215
total urls: 2927,unique urls: 2220
total urls: 2935,unique urls: 2227
total urls: 2942,unique urls: 2233
total urls: 2950,unique urls: 2240
total urls: 2957,unique urls: 2247
total urls: 2966,unique urls: 2256
total urls: 2976,unique urls: 2266
total urls: 2986,unique urls: 2276
total urls: 2996,unique urls: 2286
total urls: 3004,unique urls: 2294
total urls: 3012,unique urls: 2302
total urls: 3022,unique urls: 2311
total urls: 3031,unique urls: 2320
total urls: 3041,unique urls: 2330
total urls: 3049,unique urls: 2338
total urls: 3057,unique urls: 2346
total urls: 3067,unique urls: 2356
total urls: 3077,unique urls: 2366
total urls: 3087,unique urls: 2376
total urls: 3097,unique urls: 2386
total urls: 3107,unique urls: 2396
total urls: 3116,unique urls: 2405
total urls: 3126,unique urls: 2415
total urls: 3135,unique urls: 2424
total urls: 3145,unique urls: 2425
total urls: 3153,unique urls: 2428
total urls: 3162,unique urls: 2435
total urls: 3169,uni

In [23]:
restaurant_to_crawl = list(set(restaurant_links))  # remove duplicates

In [46]:
restaurant_to_crawl = new_url_to_crawl

In [36]:
restaurant_to_crawl[:5]

['https://www.yelp.com/biz/granville-west-hollywood-2',
 'https://www.yelp.com/biz/great-white-los-angeles-2',
 'https://www.yelp.com/biz/met-him-at-a-bar-los-angeles-3',
 'https://www.yelp.com/biz/kateen-los-angeles-2',
 'https://www.yelp.com/biz/king-chang-la-los-angeles']

In [48]:
# remember to store the urls
links_df = pd.DataFrame({'url':restaurant_to_crawl})
links_df.to_csv('current_links_0408.csv', index=False)

In [49]:
len(restaurant_to_crawl)

5381

In [41]:
get_details(restaurant_to_crawl[0])

['https://www.yelp.com/biz/granville-west-hollywood-2',
 'GRANVILLE',
 '8701 Beverly Blvd West Hollywood, CA 90048',
 '4.5 star rating',
 '2469 reviews',
 '$$',
 'American (New), Cocktail Bars',
 'Mon\n11:30 AM - 9:00 PM\nTue\n11:30 AM - 9:00 PM\nWed\n11:30 AM - 9:00 PM\nThu\n11:30 AM - 9:00 PM\nFri\n11:30 AM - 10:00 PM\nSat\n11:00 AM - 10:00 PM\nOpen now\nSun\n11:00 AM - 9:00 PM',
 'Amenities and More\nHealth ScoreA\nTakes Reservations\nOffers Delivery\nOffers Takeout\nProof of vaccination required\nAll staff fully vaccinated\nMasks required\nStaff wears masks\nVegan Options\nMany Vegetarian Options\nAccepts Credit Cards\nAccepts Android Pay\nAccepts Apple Pay\nAccepts Cryptocurrency\nOutdoor Seating\nCasual, Trendy, Romantic, Classy, Upscale\nModerate Noise\nCasual Dress\nOffers Catering\nGood for Groups\nGood For Kids\nGood for Brunch, Lunch, Dinner, Dessert\nStreet Parking, Valet Parking\nWaiter Service\nBest nights on Monday, Friday, Saturday\nHappy Hour Specials\nFull Bar\nDogs A

In [62]:
# write the detailed information to the file while downloading, in case of internet problem
count = 0
with open('restaurant_details_0408.csv', 'a', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
#     header = ['url', 'name', 'address', 'score', 'review_counts', 'price', 'category', 'open_time', 'amenities']
#     writer.writerow(header)
    
    for url in restaurant_to_crawl[3514:]:
        details = get_details(url)
        count += 1
        if details:
            writer.writerow(details)
        print(f"restaurant name: {details[1]}, {count} records downloaded")

restaurant name: Pacific Cater Truck Manufacturing, 1 records downloaded
restaurant name: Dack's Bakery Cafe, 2 records downloaded
restaurant name: P F D Cafe, 3 records downloaded
restaurant name: Birrieria el Monchis 2, 4 records downloaded
restaurant name: Mariscos El Cachi, 5 records downloaded
restaurant name: P&JDeliMarket, 6 records downloaded
restaurant name: Subway, 7 records downloaded
restaurant name: Taqueria El Viejon, 8 records downloaded
restaurant name: Culiacan Mexican Grill, 9 records downloaded
restaurant name: Tacos La Estación, 10 records downloaded
restaurant name: El Mexicano Chorizo, 11 records downloaded
restaurant name: Javier Taco's Mexico, 12 records downloaded
restaurant name: Castillo Tacos, 13 records downloaded
restaurant name: Subway, 14 records downloaded
restaurant name: La Zagala Imports, 15 records downloaded
restaurant name: In-N-Out Burger, 16 records downloaded
restaurant name: OWA, 17 records downloaded
restaurant name: Nonstop Sushi & Sake Bar,

restaurant name: El Birriazo, 144 records downloaded
restaurant name: Domino's Pizza, 145 records downloaded
restaurant name: El Faro Tacos, 146 records downloaded
restaurant name: Subway, 147 records downloaded
restaurant name: Chino's Hot-dogs, 148 records downloaded
restaurant name: Baleadas Ya, 149 records downloaded
restaurant name: San Fernando Snack Bar, 150 records downloaded
restaurant name: El Rey Del Cocktel, 151 records downloaded
restaurant name: Tacos La Esquina, 152 records downloaded
restaurant name: El Potrillo, 153 records downloaded
restaurant name: Little Caesars Pizza, 154 records downloaded
restaurant name: Del Taco, 155 records downloaded
restaurant name: Ojo De Agua, 156 records downloaded
restaurant name: Tacos Michoacán, 157 records downloaded
restaurant name: Taco Bell, 158 records downloaded
restaurant name: Gemas Mexican Food, 159 records downloaded
restaurant name: Yanelly's TACOS, 160 records downloaded
restaurant name: KFC, 161 records downloaded
restaur

restaurant name: Mr. Pete's Burger, 288 records downloaded
restaurant name: Round Table Pizza, 289 records downloaded
restaurant name: Mar Pizza, 290 records downloaded
restaurant name: McDonald's, 291 records downloaded
restaurant name: Adriana's Mexican Deli, 292 records downloaded
restaurant name: Restaurante El Buen Gusto, 293 records downloaded
restaurant name: Sea Food & Tacos Luis, 294 records downloaded
restaurant name: Guerrero Taqueria, 295 records downloaded
restaurant name: Panda Express, 296 records downloaded
restaurant name: Tacos El Ruizeñor, 297 records downloaded
restaurant name: Tacos Mexico, 298 records downloaded
restaurant name: Mariscos El Camarón Tomatero, 299 records downloaded
restaurant name: Ichiban Sushi, 300 records downloaded
restaurant name: Quemada Tacos, 301 records downloaded
restaurant name: Home Run Pizza, 302 records downloaded
restaurant name: Sunny & Fine's Breakfast Burritos, 303 records downloaded
restaurant name: Gio Fixture, 304 records downl

restaurant name: Stone Oven, 430 records downloaded
restaurant name: L'Ardente, 431 records downloaded
restaurant name: The Smith, 432 records downloaded
restaurant name: The Hamilton, 433 records downloaded
restaurant name: Boqueria Penn Quarter, 434 records downloaded
restaurant name: Unconventional Diner, 435 records downloaded
restaurant name: The Bazaar, 436 records downloaded
restaurant name: dLeña, 437 records downloaded
restaurant name: Proper 21, 438 records downloaded
restaurant name: Old Ebbitt Grill, 439 records downloaded
restaurant name: Baan Siam, 440 records downloaded
restaurant name: City Tap House Penn Quarter, 441 records downloaded
restaurant name: Succotash Prime, 442 records downloaded
restaurant name: Modena, 443 records downloaded
restaurant name: Gypsy Kitchen DC, 444 records downloaded
restaurant name: Silver Lyan, 445 records downloaded
restaurant name: Rumi’s Kitchen, 446 records downloaded
restaurant name: Rasika, 447 records downloaded
restaurant name: Im

restaurant name: Pow Pow, 577 records downloaded
restaurant name: Morris American Bar, 578 records downloaded
restaurant name: Spice 6 Modern Indian, 579 records downloaded
restaurant name: Jaleo by José Andrés, 580 records downloaded
restaurant name: The Grill, 581 records downloaded
restaurant name: West Wing Cafe, 582 records downloaded
restaurant name: Avenue Grill, 583 records downloaded
restaurant name: Maiz 64, 584 records downloaded
restaurant name: Urban Roast, 585 records downloaded
restaurant name: The Oceanaire Seafood Room, 586 records downloaded
restaurant name: Harry's Restaurant, 587 records downloaded
restaurant name: Moon Rabbit, 588 records downloaded
restaurant name: New Big Wong Chinese Restaurant, 589 records downloaded
restaurant name: Nicoletta Italian Kitchen, 590 records downloaded
restaurant name: JINYA Ramen Bar - Washington DC, 591 records downloaded
restaurant name: Del Mar, 592 records downloaded
restaurant name: Mariscos 1133, 593 records downloaded
rest

...

restaurant name: Tatte Bakery & Cafe, 1295 records downloaded
restaurant name: Ama Ami, 1296 records downloaded
restaurant name: Roti, 1297 records downloaded
restaurant name: Exiles Bar, 1298 records downloaded
restaurant name: Solace Outpost, 1299 records downloaded
restaurant name: Sweet Lemon Cafe, 1300 records downloaded
restaurant name: Bourbon Steak Washington DC, 1301 records downloaded
restaurant name: Great Wall Szechuan House, 1302 records downloaded
restaurant name: Ammathar, 1303 records downloaded
restaurant name: &pizza - Dupont, 1304 records downloaded
restaurant name: Compliments Only, 1305 records downloaded
restaurant name: My Own Pizza, 1306 records downloaded
restaurant name: Laliguras, 1307 records downloaded
restaurant name: Provision No 14, 1308 records downloaded
restaurant name: All Day by Kramers, 1309 records downloaded
restaurant name: Feta Bakery and Cafe, 1310 records downloaded
restaurant name: City Kabob and Curry House - Washington, 1311 records downlo

restaurant name: The Royce Wood-Fired Steakhouse, 1435 records downloaded
restaurant name: Communal Food & Drink, 1436 records downloaded
restaurant name: Bistro de la Gare, 1437 records downloaded
restaurant name: Shiro, 1438 records downloaded
restaurant name: The Mixx Pasadena, 1439 records downloaded
restaurant name: Zankou Chicken, 1440 records downloaded
restaurant name: Mike & Anne's Restaurant, 1441 records downloaded
restaurant name: Smosh Town, 1442 records downloaded
restaurant name: Checker Hall, 1443 records downloaded
restaurant name: Matsuri, 1444 records downloaded
restaurant name: Katsu Sando, 1445 records downloaded
restaurant name: Smitty's Grill, 1446 records downloaded
restaurant name: The Terrace, 1447 records downloaded
restaurant name: Sushi Karen, 1448 records downloaded
restaurant name: Rocco's Tavern, 1449 records downloaded
restaurant name: The Athenaeum, 1450 records downloaded
restaurant name: Arroyo Chop House, 1451 records downloaded
restaurant name: Ann

restaurant name: The Cheesecake Factory - Pasadena, 1573 records downloaded
restaurant name: Drips Cheesecake, 1574 records downloaded
restaurant name: The Old Sasoon Bakery, 1575 records downloaded
restaurant name: Tasty Choice, 1576 records downloaded
restaurant name: Barney's Only Place In Town, 1577 records downloaded
restaurant name: Suriya Thai, 1578 records downloaded
restaurant name: Wok Master, 1579 records downloaded
restaurant name: Hook Burger, 1580 records downloaded
restaurant name: Shakers, 1581 records downloaded
restaurant name: Bb.q Chicken - Pasadena, 1582 records downloaded
restaurant name: Akbar Cuisine of India, 1583 records downloaded
restaurant name: Veggie Grill, 1584 records downloaded
restaurant name: Tacos El Pollo, 1585 records downloaded
restaurant name: Burgerlords - Highland Park, 1586 records downloaded
restaurant name: Patio Grill, 1587 records downloaded
restaurant name: El Cholo Cafe, 1588 records downloaded
restaurant name: Winchell's Donuts, 1589 r

restaurant name: Los Angeles Pizzeria Company, 1710 records downloaded
restaurant name: Raffi's Catering and Banquet, 1711 records downloaded
restaurant name: Wonder Burgers, 1712 records downloaded
restaurant name: Oba Sushi Thai Tapas, 1713 records downloaded
restaurant name: Kingburg Kitchen, 1714 records downloaded
restaurant name: The Bucket, 1715 records downloaded
restaurant name: Kim Ky Noodle House, 1716 records downloaded
restaurant name: Cookies n Cream, 1717 records downloaded
restaurant name: Rosarito Mexican Food, 1718 records downloaded
restaurant name: Pho King Ngoc, 1719 records downloaded
restaurant name: IHOP, 1720 records downloaded
restaurant name: Blue Fin Sushi Roll, 1721 records downloaded
restaurant name: Lucky Baldwins Delirium & Cafe, 1722 records downloaded
restaurant name: Birria Pa La Cruda, 1723 records downloaded
restaurant name: Four Cafe, 1724 records downloaded
restaurant name: Tacos Super Gallito, 1725 records downloaded
restaurant name: Viva Tacos L

restaurant name: Uncle Tetsu, 1847 records downloaded
restaurant name: B2O Bar & Grill, 1848 records downloaded
restaurant name: Flame Broiler, 1849 records downloaded
restaurant name: McDonald's, 1850 records downloaded
restaurant name: Wah Sing Dim Sum, 1851 records downloaded
restaurant name: Moo's Craft Barbecue, 1852 records downloaded
restaurant name: bb.q Chicken - Temple City, 1853 records downloaded
restaurant name: Drive Thru Foodie Fest - Rose Bowl, 1854 records downloaded
restaurant name: Pumpernickel Deli, 1855 records downloaded
restaurant name: Mako Bowl, 1856 records downloaded
restaurant name: Barbara's At The Brewery, 1857 records downloaded
restaurant name: Kai Ramen Alhambra, 1858 records downloaded
restaurant name: Stoned + Hungry, 1859 records downloaded
restaurant name: York Kabob, 1860 records downloaded
restaurant name: La Estrella Restaurant, 1861 records downloaded
restaurant name: Tamales "ara", 1862 records downloaded
restaurant name: Kai Ramen - Arcadia, 1

In [59]:
795+2212+506

3513

In [61]:
# restart from 796th restaurant
restaurant_to_crawl[3514]

'https://www.yelp.com/biz/pacific-cater-truck-manufacturing-montebello'

In [51]:
len(restaurant_to_crawl)

5381

In [73]:
restaurant_to_crawl[1718]

'https://www.yelp.com/biz/saigon-brothers-woodland-hills-san-fernando-valley?osq=Restaurants'

In [63]:
data_df1 = pd.read_csv('restaurant_details_0408.csv',index_col=False)
# data_df2 = pd.read_csv('restaurants_details_updated_0313_latest.csv',index_col=False)

In [64]:
data_all = data_df1

In [65]:
len(data_df1)

5381

In [66]:
data_all = data_all[data_all['url']!='url']

In [67]:
data_all = data_all.drop_duplicates(subset=['url'])
len(data_all)
# data_df2 = data_df2.drop_duplicates()

5381

In [21]:
# data_df1.columns = ['url', 'name', 'address', 'score', 'review_counts', 'price', 'category', 'open_time', 'amenities']

In [94]:
# data_all.reset_index(drop=True)
data_df2 = pd.read_csv('restaurants_details_updated_0320_latest_update.csv',index_col=False)

In [103]:
data_df2.columns

Index(['Unnamed: 0', 'url', 'name', 'address', 'score', 'review_counts',
       'price', 'category', 'open_time', 'amenities'],
      dtype='object')

In [105]:
data_df2 = data_df2.iloc[:7119].drop(columns=['Unnamed: 0'])
data_df2

,url,name,address,score,review_counts,price,category,open_time,amenities
0,https://www.yelp.com/biz/plan-check-kitchen-ba...,Plan Check Kitchen + Bar,"1800 Sawtelle Blvd Los Angeles, CA 90025",4,2765 reviews,$$,"Bars, Gastropubs, American (Traditional)",Mon\r\n11:30 AM - 9:00 PM\r\nTue\r\n11:30 AM -...,Amenities and More\r\nHealth ScoreA\r\nOffers ...
1,https://www.yelp.com/biz/justin-quesos-tex-mex...,Justin Queso's Tex-Mex Restaurant & Bar,"8917 W Sunset Blvd West Hollywood, CA 90069",4,274 reviews,$$,"Mexican, Tex-Mex, Bars",Mon\r\n11:30 AM - 12:00 AM (Next day)\r\nTue\r...,Amenities and More\r\nHealth ScoreA\r\nTakes R...
2,https://www.yelp.com/biz/angelini-alimentari-l...,Angelini Alimentari,"7317 Beverly Blvd Los Angeles, CA 90036",4.5,89 reviews,$$,"Breakfast & Brunch, Salad, Italian",Mon\r\n11:00 AM - 10:00 PM\r\nTue\r\n11:00 AM ...,Amenities and More\r\nHealth ScoreA\r\nTakes R...
3,https://www.yelp.com/biz/c-and-o-trattoria-mar...,C&O Trattoria,"3016 Washington Blvd Marina del Rey, CA 90292",4,3736 reviews,$$,"Italian, Breakfast & Brunch, Wine Bars",Mon\r\n11:30 AM - 9:00 PM\r\nTue\r\n11:30 AM -...,Amenities and More\r\nTakes Reservations\r\nOf...
4,https://www.yelp.com/biz/sunny-grill-los-angel...,Sunny Grill,"12035 Venice Blvd Los Angeles, CA 90066",4,371 reviews,$$,"Salad, Burgers, Mexican",Mon\r\n10:00 AM - 10:00 PM\r\nTue\r\n10:00 AM ...,Amenities and More\r\nHealth ScoreA\r\nOffers ...
...,...,...,...,...,...,...,...,...,...
7114,https://www.yelp.com/biz/jdk-juicy-dumpling-ki...,JDK Juicy Dumpling Kitchen,NaN,4.5,167 reviews,$$,"Noodles, Ramen, Bubble Tea",Mon\n10:30 AM - 7:30 PM\nOpen now\nTue\n10:30 ...,Amenities and More\nHealth ScoreREINSPECTION D...
7115,https://www.yelp.com/biz/le-doner-kebab-los-an...,Le Doner Kebab,"6863 La Tijera Blvd Los Angeles, CA 90045",5,7 reviews,NaN,"Kebab, Pop-Up Restaurants, Modern European",Mon\nClosed\nClosed now\nTue\nClosed\nWed\nClo...,Amenities and More\nOffers Takeout\nNo Deliver...
7116,https://www.yelp.com/biz/mochinut-eagle-rock-l...,Mochinut - Eagle Rock,"3756 W Ave 40 Ste 2A Los Angeles, CA 90065",4.5,77 reviews,$,"Donuts, Hot Dogs",Mon\nClosed\nClosed now\nTue\n9:00 AM - 6:00 P...,Amenities and More\nHealth ScoreA\nOffers Deli...
7117,https://www.yelp.com/biz/salad-bar-los-angeles...,Salad Bar,NaN,3.5,41 reviews,$,"Juice Bars & Smoothies, Soup, Sandwiches",NaN,Amenities and More\nHealth ScoreB\nOffers Take...


In [68]:
data_all

,url,name,address,score,review_counts,price,category,open_time,amenities
0,https://www.yelp.com/biz/tonchin-la-los-angeles,Tonchin LA,"5665 Melrose Ave Los Angeles, CA 90038",4.5 star rating,114 reviews,$$,Japanese,Mon\nClosed\nTue\n11:30 AM - 2:00 PM\n5:30 PM ...,Amenities and More\nHealth ScoreA\nTakes Reser...
1,https://www.yelp.com/biz/broken-mouth-lees-hom...,BROKEN MOUTH | Lee's Homestyle,"718 S Los Angeles St Los Angeles, CA 90014",4.8 star rating,(1.5k reviews),$$,"Hawaiian, Korean, Comfort Food",Mon\nClosed\nTue\n11:00 AM - 2:00 PM\n4:00 PM ...,Amenities and More\nHealth ScoreA\nOffers Deli...
2,https://www.yelp.com/biz/wood-los-angeles-8,Wood,"2861 Sunset Blvd Los Angeles, CA 90026",4 star rating,1017 reviews,$$,"Pizza, Italian, Wine Bars",Mon\n11:00 AM - 11:00 PM\nTue\n11:00 AM - 11:0...,Amenities and More\nTakes Reservations\nOffers...
3,https://www.yelp.com/biz/jail-joa-los-angeles,Jail Joa,"2500 W 8th St Ste 180 Los Angeles, CA 90057",4.6 star rating,(135 reviews),$$,"Gastropubs, Korean, Asian Fusion",Mon\n6:00 PM - 2:00 AM (Next day)\nTue\n6:00 P...,Amenities and More\nOffers Delivery\nOffers Ta...
4,https://www.yelp.com/biz/majordomo-los-angeles-2,Majordomo,"1725 Naud St Los Angeles, CA 90012",4.2 star rating,(1.3k reviews),$$$,"American (New), Asian Fusion, Korean",Mon\n5:00 PM - 10:00 PM\nTue\n5:00 PM - 10:00 ...,Amenities and More\nHealth ScoreA\nTakes Reser...
...,...,...,...,...,...,...,...,...,...
5376,https://www.yelp.com/biz/kai-ramen-arcadia-arc...,Kai Ramen - Arcadia,"1130 S Baldwin Ave Ste 6 Arcadia, CA 91007",4.3 star rating,(469 reviews),$$,"Ramen, Noodles",Mon\n12:00 PM - 10:30 PM\nTue\n12:00 PM - 10:3...,Amenities and More\nHealth ScoreA\nOffers Deli...
5377,https://www.yelp.com/biz/el-huarache-azteca-lo...,El Huarache Azteca,"5225 York Blvd Los Angeles, CA 90042",3.7 star rating,(996 reviews),$$,"Mexican, Breakfast & Brunch",Mon\n8:00 AM - 8:00 PM\nTue\n8:00 AM - 8:00 PM...,Amenities and More\nOffers Delivery\nNo Reserv...
5378,https://www.yelp.com/biz/louisiana-fried-chick...,Louisiana Fried Chicken,"1515 N Lake Ave Pasadena, CA 91104",4 star rating,132 reviews,$,"Cajun/Creole, Chicken Wings",Mon\nClosed\nTue\n11:00 AM - 8:00 PM\nWed\n11:...,Amenities and More\nHealth Score96 out of 100\...
5379,https://www.yelp.com/biz/cheeky-cheeky-bun-bun...,Cheeky Cheeky Bun Bun,"627 W Duarte Rd Arcadia, CA 91007",3.8 star rating,(19 reviews),NaN,Dim Sum,Mon\n11:00 AM - 9:30 PM\nTue\n11:00 AM - 9:30 ...,Amenities and More\nTakes Reservations\nOffers...


In [69]:
def clean_rating(x):
    # clean the "score"
    if 'star rating' in x:
        x = x.strip(' star rating')
    else:
        x = ''
    return x

In [70]:
data_all['score'] = data_all['score'].apply(lambda x:str(x).strip(' star rating'))
data_all['score'].value_counts()

4      940
4.5    665
3.5    484
5      420
       398
3      301
2.5    165
2      158
3.9    153
4.3    133
4.2    131
3.8    131
4.1    126
4.4    119
3.7    113
4.6     88
3.4     78
3.6     72
4.7     70
3.3     67
1.5     67
4.8     52
1       50
3.2     43
2.8     40
3.1     35
2.9     29
2.6     28
2.3     28
2.7     28
2.4     24
1.9     24
1.8     21
4.9     21
2.1     20
1.7     18
2.2     14
1.6     11
1.4      7
1.3      7
1.2      2
Name: score, dtype: int64

In [71]:
data_all.head()

,url,name,address,score,review_counts,price,category,open_time,amenities
0,https://www.yelp.com/biz/tonchin-la-los-angeles,Tonchin LA,"5665 Melrose Ave Los Angeles, CA 90038",4.5,114 reviews,$$,Japanese,Mon\nClosed\nTue\n11:30 AM - 2:00 PM\n5:30 PM ...,Amenities and More\nHealth ScoreA\nTakes Reser...
1,https://www.yelp.com/biz/broken-mouth-lees-hom...,BROKEN MOUTH | Lee's Homestyle,"718 S Los Angeles St Los Angeles, CA 90014",4.8,(1.5k reviews),$$,"Hawaiian, Korean, Comfort Food",Mon\nClosed\nTue\n11:00 AM - 2:00 PM\n4:00 PM ...,Amenities and More\nHealth ScoreA\nOffers Deli...
2,https://www.yelp.com/biz/wood-los-angeles-8,Wood,"2861 Sunset Blvd Los Angeles, CA 90026",4,1017 reviews,$$,"Pizza, Italian, Wine Bars",Mon\n11:00 AM - 11:00 PM\nTue\n11:00 AM - 11:0...,Amenities and More\nTakes Reservations\nOffers...
3,https://www.yelp.com/biz/jail-joa-los-angeles,Jail Joa,"2500 W 8th St Ste 180 Los Angeles, CA 90057",4.6,(135 reviews),$$,"Gastropubs, Korean, Asian Fusion",Mon\n6:00 PM - 2:00 AM (Next day)\nTue\n6:00 P...,Amenities and More\nOffers Delivery\nOffers Ta...
4,https://www.yelp.com/biz/majordomo-los-angeles-2,Majordomo,"1725 Naud St Los Angeles, CA 90012",4.2,(1.3k reviews),$$$,"American (New), Asian Fusion, Korean",Mon\n5:00 PM - 10:00 PM\nTue\n5:00 PM - 10:00 ...,Amenities and More\nHealth ScoreA\nTakes Reser...


In [124]:
df_combine = pd.concat([data_df2, data_all], axis=0)
df_combine

,url,name,address,score,review_counts,price,category,open_time,amenities
0,https://www.yelp.com/biz/plan-check-kitchen-ba...,Plan Check Kitchen + Bar,"1800 Sawtelle Blvd Los Angeles, CA 90025",4,2765 reviews,$$,"Bars, Gastropubs, American (Traditional)",Mon\r\n11:30 AM - 9:00 PM\r\nTue\r\n11:30 AM -...,Amenities and More\r\nHealth ScoreA\r\nOffers ...
1,https://www.yelp.com/biz/justin-quesos-tex-mex...,Justin Queso's Tex-Mex Restaurant & Bar,"8917 W Sunset Blvd West Hollywood, CA 90069",4,274 reviews,$$,"Mexican, Tex-Mex, Bars",Mon\r\n11:30 AM - 12:00 AM (Next day)\r\nTue\r...,Amenities and More\r\nHealth ScoreA\r\nTakes R...
2,https://www.yelp.com/biz/angelini-alimentari-l...,Angelini Alimentari,"7317 Beverly Blvd Los Angeles, CA 90036",4.5,89 reviews,$$,"Breakfast & Brunch, Salad, Italian",Mon\r\n11:00 AM - 10:00 PM\r\nTue\r\n11:00 AM ...,Amenities and More\r\nHealth ScoreA\r\nTakes R...
3,https://www.yelp.com/biz/c-and-o-trattoria-mar...,C&O Trattoria,"3016 Washington Blvd Marina del Rey, CA 90292",4,3736 reviews,$$,"Italian, Breakfast & Brunch, Wine Bars",Mon\r\n11:30 AM - 9:00 PM\r\nTue\r\n11:30 AM -...,Amenities and More\r\nTakes Reservations\r\nOf...
4,https://www.yelp.com/biz/sunny-grill-los-angel...,Sunny Grill,"12035 Venice Blvd Los Angeles, CA 90066",4,371 reviews,$$,"Salad, Burgers, Mexican",Mon\r\n10:00 AM - 10:00 PM\r\nTue\r\n10:00 AM ...,Amenities and More\r\nHealth ScoreA\r\nOffers ...
...,...,...,...,...,...,...,...,...,...
2747,https://www.yelp.com/biz/cheebo-burbank?osq=Re...,Cheebo,"271 E Orange Grove Ave Burbank, CA 91502",5,2 reviews,NaN,"Salad, Burgers, Sandwiches",Mon\n8:00 AM - 2:00 AM (Next day)\nTue\n8:00 A...,NaN
2748,https://www.yelp.com/biz/dish-la-canada-flintr...,Dish,"734 Foothill Blvd La Canada Flintridge, CA 91011",3.5,890 reviews,$$,"American (Traditional), Comfort Food",Mon\n8:00 AM - 8:00 PM\nTue\n8:00 AM - 8:00 PM...,Amenities and More\nTakes Reservations\nOffers...
2749,https://www.yelp.com/biz/phillys-best-burbank?...,Philly's Best,"1419 W Olive Ave Burbank, CA 91506",3.5,566 reviews,$$,Cheesesteaks,Mon\n11:00 AM - 9:00 PM\nTue\n11:00 AM - 9:00 ...,Amenities and More\nHealth ScoreA\nOffers Deli...
2750,https://www.yelp.com/biz/kalaveras-simi-valley...,NaN,"1213 Simi Town Center Way Simi Valley, CA 93065",3.5,71 reviews,$$$,"Cocktail Bars, Breakfast & Brunch, Mexican",NaN,Amenities and More\nTakes Reservations\nCasual...


In [73]:
data_all = data_all.drop_duplicates(subset=['url']).reset_index(drop=True)

In [74]:
data_all

,url,name,address,score,review_counts,price,category,open_time,amenities
0,https://www.yelp.com/biz/tonchin-la-los-angeles,Tonchin LA,"5665 Melrose Ave Los Angeles, CA 90038",4.5,114 reviews,$$,Japanese,Mon\nClosed\nTue\n11:30 AM - 2:00 PM\n5:30 PM ...,Amenities and More\nHealth ScoreA\nTakes Reser...
1,https://www.yelp.com/biz/broken-mouth-lees-hom...,BROKEN MOUTH | Lee's Homestyle,"718 S Los Angeles St Los Angeles, CA 90014",4.8,(1.5k reviews),$$,"Hawaiian, Korean, Comfort Food",Mon\nClosed\nTue\n11:00 AM - 2:00 PM\n4:00 PM ...,Amenities and More\nHealth ScoreA\nOffers Deli...
2,https://www.yelp.com/biz/wood-los-angeles-8,Wood,"2861 Sunset Blvd Los Angeles, CA 90026",4,1017 reviews,$$,"Pizza, Italian, Wine Bars",Mon\n11:00 AM - 11:00 PM\nTue\n11:00 AM - 11:0...,Amenities and More\nTakes Reservations\nOffers...
3,https://www.yelp.com/biz/jail-joa-los-angeles,Jail Joa,"2500 W 8th St Ste 180 Los Angeles, CA 90057",4.6,(135 reviews),$$,"Gastropubs, Korean, Asian Fusion",Mon\n6:00 PM - 2:00 AM (Next day)\nTue\n6:00 P...,Amenities and More\nOffers Delivery\nOffers Ta...
4,https://www.yelp.com/biz/majordomo-los-angeles-2,Majordomo,"1725 Naud St Los Angeles, CA 90012",4.2,(1.3k reviews),$$$,"American (New), Asian Fusion, Korean",Mon\n5:00 PM - 10:00 PM\nTue\n5:00 PM - 10:00 ...,Amenities and More\nHealth ScoreA\nTakes Reser...
...,...,...,...,...,...,...,...,...,...
5376,https://www.yelp.com/biz/kai-ramen-arcadia-arc...,Kai Ramen - Arcadia,"1130 S Baldwin Ave Ste 6 Arcadia, CA 91007",4.3,(469 reviews),$$,"Ramen, Noodles",Mon\n12:00 PM - 10:30 PM\nTue\n12:00 PM - 10:3...,Amenities and More\nHealth ScoreA\nOffers Deli...
5377,https://www.yelp.com/biz/el-huarache-azteca-lo...,El Huarache Azteca,"5225 York Blvd Los Angeles, CA 90042",3.7,(996 reviews),$$,"Mexican, Breakfast & Brunch",Mon\n8:00 AM - 8:00 PM\nTue\n8:00 AM - 8:00 PM...,Amenities and More\nOffers Delivery\nNo Reserv...
5378,https://www.yelp.com/biz/louisiana-fried-chick...,Louisiana Fried Chicken,"1515 N Lake Ave Pasadena, CA 91104",4,132 reviews,$,"Cajun/Creole, Chicken Wings",Mon\nClosed\nTue\n11:00 AM - 8:00 PM\nWed\n11:...,Amenities and More\nHealth Score96 out of 100\...
5379,https://www.yelp.com/biz/cheeky-cheeky-bun-bun...,Cheeky Cheeky Bun Bun,"627 W Duarte Rd Arcadia, CA 91007",3.8,(19 reviews),NaN,Dim Sum,Mon\n11:00 AM - 9:30 PM\nTue\n11:00 AM - 9:30 ...,Amenities and More\nTakes Reservations\nOffers...


In [75]:
data_all.to_csv('restaurants_details_updated_0409_latest_update.csv',encoding='utf-8')

In [26]:
# len(data_df2)

5191

In [27]:
# combine the newly downloaded data and the previous downloaded
# data_all = pd.concat([data_df2, data_df1], ignore_index=True)

In [28]:
# data_all = data_all.drop_duplicates()

In [29]:
# data_all.tail()

,url,name,address,score,review_counts,price,category,open_time,amenities
7114,https://www.yelp.com/biz/jdk-juicy-dumpling-ki...,JDK Juicy Dumpling Kitchen,NaN,4.5 star rating,167 reviews,$$,"Noodles, Ramen, Bubble Tea",Mon\n10:30 AM - 7:30 PM\nOpen now\nTue\n10:30 ...,Amenities and More\nHealth ScoreREINSPECTION D...
7115,https://www.yelp.com/biz/le-doner-kebab-los-an...,Le Doner Kebab,"6863 La Tijera Blvd Los Angeles, CA 90045",5 star rating,7 reviews,NaN,"Kebab, Pop-Up Restaurants, Modern European",Mon\nClosed\nClosed now\nTue\nClosed\nWed\nClo...,Amenities and More\nOffers Takeout\nNo Deliver...
7116,https://www.yelp.com/biz/mochinut-eagle-rock-l...,Mochinut - Eagle Rock,"3756 W Ave 40 Ste 2A Los Angeles, CA 90065",4.5 star rating,77 reviews,$,"Donuts, Hot Dogs",Mon\nClosed\nClosed now\nTue\n9:00 AM - 6:00 P...,Amenities and More\nHealth ScoreA\nOffers Deli...
7117,https://www.yelp.com/biz/salad-bar-los-angeles...,Salad Bar,NaN,3.5 star rating,41 reviews,$,"Juice Bars & Smoothies, Soup, Sandwiches",NaN,Amenities and More\nHealth ScoreB\nOffers Take...
7118,https://www.yelp.com/biz/the-boba-truck-northr...,The Boba Truck,NaN,4.5 star rating,668 reviews,$,"Taiwanese, Bubble Tea, Food Trucks",Mon\n11:00 AM - 8:00 PM\nOpen now\nTue\n11:00 ...,Amenities and More\nHealth ScoreA\nOffers Deli...


In [30]:
# len(data_all)

7119

In [31]:
# data_all[data_all['url']=='url']

,url,name,address,score,review_counts,price,category,open_time,amenities


In [32]:
# data_all = data_all[data_all['url']!='url']

In [33]:
# data_all = data_all.drop_duplicates(subset=['url'])

In [34]:
# len(data_all)

7113

In [35]:
# data_all

,url,name,address,score,review_counts,price,category,open_time,amenities
0,https://www.yelp.com/biz/plan-check-kitchen-ba...,Plan Check Kitchen + Bar,"1800 Sawtelle Blvd Los Angeles, CA 90025",4.0,2765 reviews,$$,"Bars, Gastropubs, American (Traditional)",Mon\r\n11:30 AM - 9:00 PM\r\nTue\r\n11:30 AM -...,Amenities and More\r\nHealth ScoreA\r\nOffers ...
1,https://www.yelp.com/biz/justin-quesos-tex-mex...,Justin Queso's Tex-Mex Restaurant & Bar,"8917 W Sunset Blvd West Hollywood, CA 90069",4.0,274 reviews,$$,"Mexican, Tex-Mex, Bars",Mon\r\n11:30 AM - 12:00 AM (Next day)\r\nTue\r...,Amenities and More\r\nHealth ScoreA\r\nTakes R...
2,https://www.yelp.com/biz/angelini-alimentari-l...,Angelini Alimentari,"7317 Beverly Blvd Los Angeles, CA 90036",4.5,89 reviews,$$,"Breakfast & Brunch, Salad, Italian",Mon\r\n11:00 AM - 10:00 PM\r\nTue\r\n11:00 AM ...,Amenities and More\r\nHealth ScoreA\r\nTakes R...
3,https://www.yelp.com/biz/c-and-o-trattoria-mar...,C&O Trattoria,"3016 Washington Blvd Marina del Rey, CA 90292",4.0,3736 reviews,$$,"Italian, Breakfast & Brunch, Wine Bars",Mon\r\n11:30 AM - 9:00 PM\r\nTue\r\n11:30 AM -...,Amenities and More\r\nTakes Reservations\r\nOf...
4,https://www.yelp.com/biz/sunny-grill-los-angel...,Sunny Grill,"12035 Venice Blvd Los Angeles, CA 90066",4.0,371 reviews,$$,"Salad, Burgers, Mexican",Mon\r\n10:00 AM - 10:00 PM\r\nTue\r\n10:00 AM ...,Amenities and More\r\nHealth ScoreA\r\nOffers ...
...,...,...,...,...,...,...,...,...,...
7114,https://www.yelp.com/biz/jdk-juicy-dumpling-ki...,JDK Juicy Dumpling Kitchen,NaN,4.5 star rating,167 reviews,$$,"Noodles, Ramen, Bubble Tea",Mon\n10:30 AM - 7:30 PM\nOpen now\nTue\n10:30 ...,Amenities and More\nHealth ScoreREINSPECTION D...
7115,https://www.yelp.com/biz/le-doner-kebab-los-an...,Le Doner Kebab,"6863 La Tijera Blvd Los Angeles, CA 90045",5 star rating,7 reviews,NaN,"Kebab, Pop-Up Restaurants, Modern European",Mon\nClosed\nClosed now\nTue\nClosed\nWed\nClo...,Amenities and More\nOffers Takeout\nNo Deliver...
7116,https://www.yelp.com/biz/mochinut-eagle-rock-l...,Mochinut - Eagle Rock,"3756 W Ave 40 Ste 2A Los Angeles, CA 90065",4.5 star rating,77 reviews,$,"Donuts, Hot Dogs",Mon\nClosed\nClosed now\nTue\n9:00 AM - 6:00 P...,Amenities and More\nHealth ScoreA\nOffers Deli...
7117,https://www.yelp.com/biz/salad-bar-los-angeles...,Salad Bar,NaN,3.5 star rating,41 reviews,$,"Juice Bars & Smoothies, Soup, Sandwiches",NaN,Amenities and More\nHealth ScoreB\nOffers Take...


In [40]:
# data_all = data_all[data_all['score']!='score']

In [57]:
# data_all = data_all.drop_duplicates()

In [33]:
len(data_all)

2751

In [34]:
data_all = data_all.reset_index(drop=True)

In [37]:
data_all['url'].duplicated().sum()

0

In [35]:
data_all.to_csv('restaurants_details_updated_0325_update.csv',encoding='utf-8')

In [62]:
df = pd.read_csv('restaurants_details_updated_0325_latest_update.csv')

In [63]:
df

,Unnamed: 0,url,name,address,score,review_counts,price,category,open_time,amenities
0,0,https://www.yelp.com/biz/plan-check-kitchen-ba...,Plan Check Kitchen + Bar,"1800 Sawtelle Blvd Los Angeles, CA 90025",4,2765 reviews,$$,"Bars, Gastropubs, American (Traditional)",Mon\r\n11:30 AM - 9:00 PM\r\nTue\r\n11:30 AM -...,Amenities and More\r\nHealth ScoreA\r\nOffers ...
1,1,https://www.yelp.com/biz/justin-quesos-tex-mex...,Justin Queso's Tex-Mex Restaurant & Bar,"8917 W Sunset Blvd West Hollywood, CA 90069",4,274 reviews,$$,"Mexican, Tex-Mex, Bars",Mon\r\n11:30 AM - 12:00 AM (Next day)\r\nTue\r...,Amenities and More\r\nHealth ScoreA\r\nTakes R...
2,2,https://www.yelp.com/biz/angelini-alimentari-l...,Angelini Alimentari,"7317 Beverly Blvd Los Angeles, CA 90036",4.5,89 reviews,$$,"Breakfast & Brunch, Salad, Italian",Mon\r\n11:00 AM - 10:00 PM\r\nTue\r\n11:00 AM ...,Amenities and More\r\nHealth ScoreA\r\nTakes R...
3,3,https://www.yelp.com/biz/c-and-o-trattoria-mar...,C&O Trattoria,"3016 Washington Blvd Marina del Rey, CA 90292",4,3736 reviews,$$,"Italian, Breakfast & Brunch, Wine Bars",Mon\r\n11:30 AM - 9:00 PM\r\nTue\r\n11:30 AM -...,Amenities and More\r\nTakes Reservations\r\nOf...
4,4,https://www.yelp.com/biz/sunny-grill-los-angel...,Sunny Grill,"12035 Venice Blvd Los Angeles, CA 90066",4,371 reviews,$$,"Salad, Burgers, Mexican",Mon\r\n10:00 AM - 10:00 PM\r\nTue\r\n10:00 AM ...,Amenities and More\r\nHealth ScoreA\r\nOffers ...
...,...,...,...,...,...,...,...,...,...,...
9859,9859,https://www.yelp.com/biz/cheebo-burbank?osq=Re...,Cheebo,NaN,5,2 reviews,NaN,"Salad, Burgers, Sandwiches",Mon\n8:00 AM - 2:00 AM (Next day)\nTue\n8:00 A...,NaN
9860,9860,https://www.yelp.com/biz/dish-la-canada-flintr...,Dish,NaN,3.5,891 reviews,$$,"American (Traditional), Comfort Food",Mon\n8:00 AM - 8:00 PM\nTue\n8:00 AM - 8:00 PM...,Amenities and More\nTakes Reservations\nOffers...
9861,9861,https://www.yelp.com/biz/phillys-best-burbank?...,Philly's Best,NaN,3.5,566 reviews,$$,Cheesesteaks,Mon\n11:00 AM - 9:00 PM\nTue\n11:00 AM - 9:00 ...,Amenities and More\nHealth ScoreA\nOffers Deli...
9862,9862,https://www.yelp.com/biz/kalaveras-simi-valley...,Kalaveras,NaN,3.5,70 reviews,$$$,"Cocktail Bars, Breakfast & Brunch, Mexican",NaN,Amenities and More\nTakes Reservations\nCasual...


In [43]:
df = df.drop_duplicates(subset=['url'])

In [61]:
len(df)

9864

In [45]:
df['url'].duplicated().sum()

0

In [52]:
df['Unnamed: 0'] = range(0, len(df))

C:\Users\61427\AppData\Local\Temp\ipykernel_2176\1078329299.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Unnamed: 0'] = range(0, len(df))


In [53]:
# df.reset_index()

,index,Unnamed: 0,url,name,address,score,review_counts,price,category,open_time,amenities
0,0,0,https://www.yelp.com/biz/plan-check-kitchen-ba...,Plan Check Kitchen + Bar,"1800 Sawtelle Blvd Los Angeles, CA 90025",4,2765 reviews,$$,"Bars, Gastropubs, American (Traditional)",Mon\r\n11:30 AM - 9:00 PM\r\nTue\r\n11:30 AM -...,Amenities and More\r\nHealth ScoreA\r\nOffers ...
1,1,1,https://www.yelp.com/biz/justin-quesos-tex-mex...,Justin Queso's Tex-Mex Restaurant & Bar,"8917 W Sunset Blvd West Hollywood, CA 90069",4,274 reviews,$$,"Mexican, Tex-Mex, Bars",Mon\r\n11:30 AM - 12:00 AM (Next day)\r\nTue\r...,Amenities and More\r\nHealth ScoreA\r\nTakes R...
2,2,2,https://www.yelp.com/biz/angelini-alimentari-l...,Angelini Alimentari,"7317 Beverly Blvd Los Angeles, CA 90036",4.5,89 reviews,$$,"Breakfast & Brunch, Salad, Italian",Mon\r\n11:00 AM - 10:00 PM\r\nTue\r\n11:00 AM ...,Amenities and More\r\nHealth ScoreA\r\nTakes R...
3,3,3,https://www.yelp.com/biz/c-and-o-trattoria-mar...,C&O Trattoria,"3016 Washington Blvd Marina del Rey, CA 90292",4,3736 reviews,$$,"Italian, Breakfast & Brunch, Wine Bars",Mon\r\n11:30 AM - 9:00 PM\r\nTue\r\n11:30 AM -...,Amenities and More\r\nTakes Reservations\r\nOf...
4,4,4,https://www.yelp.com/biz/sunny-grill-los-angel...,Sunny Grill,"12035 Venice Blvd Los Angeles, CA 90066",4,371 reviews,$$,"Salad, Burgers, Mexican",Mon\r\n10:00 AM - 10:00 PM\r\nTue\r\n10:00 AM ...,Amenities and More\r\nHealth ScoreA\r\nOffers ...
...,...,...,...,...,...,...,...,...,...,...,...
9859,9865,9859,https://www.yelp.com/biz/cheebo-burbank?osq=Re...,Cheebo,NaN,5,2 reviews,NaN,"Salad, Burgers, Sandwiches",Mon\n8:00 AM - 2:00 AM (Next day)\nTue\n8:00 A...,NaN
9860,9866,9860,https://www.yelp.com/biz/dish-la-canada-flintr...,Dish,NaN,3.5,891 reviews,$$,"American (Traditional), Comfort Food",Mon\n8:00 AM - 8:00 PM\nTue\n8:00 AM - 8:00 PM...,Amenities and More\nTakes Reservations\nOffers...
9861,9867,9861,https://www.yelp.com/biz/phillys-best-burbank?...,Philly's Best,NaN,3.5,566 reviews,$$,Cheesesteaks,Mon\n11:00 AM - 9:00 PM\nTue\n11:00 AM - 9:00 ...,Amenities and More\nHealth ScoreA\nOffers Deli...
9862,9868,9862,https://www.yelp.com/biz/kalaveras-simi-valley...,Kalaveras,NaN,3.5,70 reviews,$$$,"Cocktail Bars, Breakfast & Brunch, Mexican",NaN,Amenities and More\nTakes Reservations\nCasual...


In [80]:
crawled_df = pd.read_csv('combined_rest_health_0409_filtered.csv')
crawled_df.head()

,FACILITY_NAME,FACILITY_ADDRESS,FACILITY_CITY,FACILITY_ID,FACILITY_STATE,FACILITY_ZIP,OWNER_ID,PE_DESCRIPTION,SCORE,VIOLATION_CODE,VIOLATION_DESCRIPTION,type,size,risk_level,ID,restaurant_id,url,name,address,score,review_counts,price,category,open_time,amenities,new_address,state,zipcode,city
0,YAMAKASE,11901 SANTA MONICA BLVD # 111,LOS ANGELES,FA0258281,CA,90025,OW0250494,RESTAURANT (0-30) SEATS HIGH RISK,95.000000,"F006,F037,F058",# 06. Adequate handwashing facilities supplied...,restaurant,0-30,HIGH,13088,2267,https://www.yelp.com/biz/yamakase-los-angeles,Yamakase,"11901 Santa Monica Blvd Los Angeles, CA 90025",4.4,(270 reviews),$$$$,"Japanese, Sushi Bars",Mon\nClosed\nTue\n7:00 PM - 10:00 PM\nWed\n7:0...,Amenities and More\nTakes Reservations\nOffers...,11901 Santa Monica Blvd Los Angeles,CA,90025,Los Angeles
1,GOOD STUFF BURGERS,11903 W OLYMPIC BLVD,LOS ANGELES,FA0164067,CA,90064,OW0127530,RESTAURANT (0-30) SEATS HIGH RISK,92.472222,"F035,F034,F043,F030,F044,F001,F035,F023,F040,F...",# 35. Equipment/Utensils - approved; installed...,restaurant,0-30,HIGH,4582,2270,https://www.yelp.com/biz/good-stuff-burgers-lo...,Good Stuff Burgers,"11903 W Olympic Blvd Los Angeles, CA 90064",4.5,211 reviews,$$,"Salad, Burgers, Sandwiches",Mon\n7:00 AM - 9:00 PM\nTue\n7:00 AM - 9:00 PM...,Amenities and More\nHealth ScoreA\nOffers Deli...,11903 W Olympic Blvd Los Angeles,CA,90064,Los Angeles
2,EL TEXANITO #2,4501 S ALAMEDA ST # J7,LOS ANGELES,FA0021494,CA,90058,OW0002046,RESTAURANT (0-30) SEATS MODERATE RISK,93.875000,"F033,F035,F052,F044,F032,F036,F006,F052,F035,F...",# 33. Nonfood-contact surfaces clean and in go...,restaurant,0-30,MODER,3763,3231,https://www.yelp.com/biz/el-texanito-los-angeles,El Texanito,"4501 S Alameda St Los Angeles, CA 90058",3.0,(2 reviews),NaN,Restaurants,NaN,Amenities and More\nHealth ScoreB\nOffers Take...,4501 S Alameda St Los Angeles,CA,90058,Los Angeles
3,AERO BISTRO-EMBASSY SUITES,9801 AIRPORT BLVD,LOS ANGELES,FA0002923,CA,90045,OW0003619,RESTAURANT (61-150) SEATS HIGH RISK,93.842105,"F053,F044,F040,F039,F006,F052,F043,F040,F044,F...","# 21a. Hot Water Available,# 44. Floors, walls...",restaurant,61-150,HIGH,406,2327,https://www.yelp.com/biz/aerobistro-los-angeles,AeroBistro,9801 Airport Blvd Embassy Suites LAX N Los Ang...,3.0,32 reviews,$$,American (New),Mon\n5:00 PM - 10:00 PM\nTue\n5:00 PM - 10:00 ...,Amenities and More\nTakes Reservations\nOffers...,9801 Airport Blvd Embassy Suites LAX N Los Ang...,CA,90045,Los Angeles
4,"PESCADERIA PUERTO EL TRIUNFO, INC.",1713 E VERNON AVE,LOS ANGELES,FA0178733,CA,90058,OW0184416,"FOOD MKT RETAIL (25-1,999 SF) HIGH RISK",93.714286,"F036,F006,F023,F016,F035,F006,F043,F035,F052,F...","# 36. Equipment, utensils and linens: storage ...",food market,"25-1,999 SF",HIGH,9016,2868,https://www.yelp.com/biz/pescaderia-puerto-pen...,Pescaderia Puerto Penasco,"1713 E Vernon Ave Los Angeles, CA 90058",5.0,(2 reviews),$,Latin American,Mon\n9:00 AM - 6:00 PM\nTue\n9:00 AM - 6:00 PM...,Amenities and More\nOffers Takeout\nNo Reserva...,1713 E Vernon Ave Los Angeles,CA,90058,Los Angeles


In [83]:
rest_urls = crawled_df['url'].tolist()
rest_urls[:10]

['https://www.yelp.com/biz/yamakase-los-angeles',
 'https://www.yelp.com/biz/good-stuff-burgers-los-angeles-2',
 'https://www.yelp.com/biz/el-texanito-los-angeles',
 'https://www.yelp.com/biz/aerobistro-los-angeles',
 'https://www.yelp.com/biz/pescaderia-puerto-penasco-los-angeles',
 'https://www.yelp.com/biz/founders-room-los-angeles',
 'https://www.yelp.com/biz/mama-gallina-vernon',
 'https://www.yelp.com/biz/alex-burgers-los-angeles',
 'https://www.yelp.com/biz/homestate-los-angeles-4',
 'https://www.yelp.com/biz/andiamo-los-angeles-3']

In [85]:
# crawl the detailed information of each restaurant
driver = webdriver.Chrome(executable_path="C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe",options=options)
driver.set_window_size(1920,1080)
driver.get(rest_urls[1])
wait = WebDriverWait(driver, 1)


In [86]:
def get_rest_details(url):
    # crawl the detailed information of each restaurant
    driver = webdriver.Chrome(executable_path="C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe",options=options)
    driver.set_window_size(1920,1080)
    driver.get(url)
    wait = WebDriverWait(driver, 1)
    
    try:
        if driver.find_element('xpath','.//div[@class=" photo-header-buttons__09f24__UU4lV  border-color--default__09f24__NPAKY"]/span/a/span'):
            num_photos = driver.find_element('xpath','.//div[@class=" photo-header-buttons__09f24__UU4lV  border-color--default__09f24__NPAKY"]/span/a/span').text
    except NoSuchElementException:
        num_photos = ''
    # num_photos
    
    try:
        if driver.find_element('xpath','.//div[*[contains(text(), "Business website")]]/p[2]'):
    #         num_photos = driver.find_element('xpath','.//div[@class=" photo-header-buttons__09f24__UU4lV  border-color--default__09f24__NPAKY"]/span/a/span').text
            is_bus_web = True
    except NoSuchElementException:
        is_bus_web = False
        
    try:
        if driver.find_element('xpath','.//div[*[contains(text(), "Phone number")]]/p[2]'):
    #         num_photos = driver.find_element('xpath','.//div[@class=" photo-header-buttons__09f24__UU4lV  border-color--default__09f24__NPAKY"]/span/a/span').text
            is_phone_number = True
    except NoSuchElementException:
        is_phone_number = False
        
    try:
        if driver.find_element('xpath','.//button/span[*[contains(text(), "Message the Business")]]'):
    #         num_photos = driver.find_element('xpath','.//div[@class=" photo-header-buttons__09f24__UU4lV  border-color--default__09f24__NPAKY"]/span/a/span').text
            is_message_bus = True
    except NoSuchElementException:
        is_message_bus = False
   
    
        # .//button[@type="submit"]/span/p[contains(text(), "Attributes")]
    try:
        if driver.find_element('xpath','.//button[@type="submit"]/span/p[contains(text(), "Attribute")]'):
            num_attributes = driver.find_element('xpath','.//button[@type="submit"]/span/p[contains(text(), "Attribute")]').text.strip(' More Attributes')
    except NoSuchElementException:
        num_attributes = ''
        
    
    try:
        if driver.find_element('xpath',".//div/a[.//span[contains(.,'See all')] and .//span[contains(.,'question')]]//span"):
            num_questions = driver.find_element('xpath', ".//div/a[.//span[contains(.,'See all')] and .//span[contains(.,'question')]]//span").text
    except NoSuchElementException:
        num_questions = ''
#     num_questions
    
    try:
        if driver.find_element('xpath','.//div[@class=" review__09f24__oHr9V  border-color--default__09f24__NPAKY"]'):
            comments = driver.find_elements('xpath','.//div[@class=" review__09f24__oHr9V  border-color--default__09f24__NPAKY"]')
            comments_list = []
            for comment in comments:
                content = comment.find_element('xpath','..//p[@class="comment__09f24__gu0rG css-qgunke"]').text
    #             print(content)
                comments_list.append(content)
    except NoSuchElementException:
        comments_list = []
    # comments
 
    driver.quit()
    return [url, num_photos,is_bus_web,is_phone_number,is_message_bus,num_attributes,num_questions, comments_list]

In [87]:
len(rest_urls)

334

In [56]:
# get_rest_details(rest_urls[0])

In [90]:
options = webdriver.ChromeOptions()
options.add_argument("--headless")

In [91]:
# write the detailed information to the file while downloading, in case of internet problem
count = 0
with open('restaurant_details_append_0409.csv', 'a', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    header = ['url', 'num_photos','is_bus_web','is_phone_number','is_message_bus','num_attributes','num_questions', 'comments_list']
    writer.writerow(header)
    
    for url in rest_urls:
        details = get_rest_details(url)
        count += 1
        if details:
            writer.writerow(details)
        print(f"restaurant url: {details[0]}, {count} records downloaded")

restaurant url: https://www.yelp.com/biz/yamakase-los-angeles, 1 records downloaded
restaurant url: https://www.yelp.com/biz/good-stuff-burgers-los-angeles-2, 2 records downloaded
restaurant url: https://www.yelp.com/biz/el-texanito-los-angeles, 3 records downloaded
restaurant url: https://www.yelp.com/biz/aerobistro-los-angeles, 4 records downloaded
restaurant url: https://www.yelp.com/biz/pescaderia-puerto-penasco-los-angeles, 5 records downloaded
restaurant url: https://www.yelp.com/biz/founders-room-los-angeles, 6 records downloaded
restaurant url: https://www.yelp.com/biz/mama-gallina-vernon, 7 records downloaded
restaurant url: https://www.yelp.com/biz/alex-burgers-los-angeles, 8 records downloaded
restaurant url: https://www.yelp.com/biz/homestate-los-angeles-4, 9 records downloaded
restaurant url: https://www.yelp.com/biz/andiamo-los-angeles-3, 10 records downloaded
restaurant url: https://www.yelp.com/biz/andys-pizzeria-and-mexican-food-los-angeles, 11 records downloaded
resta

restaurant url: https://www.yelp.com/biz/daikokuya-little-tokyo-los-angeles, 89 records downloaded
restaurant url: https://www.yelp.com/biz/del-taco-los-angeles-7, 90 records downloaded
restaurant url: https://www.yelp.com/biz/delias-restaurant-los-angeles, 91 records downloaded
restaurant url: https://www.yelp.com/biz/delicias-de-el-salvador-los-angeles, 92 records downloaded
restaurant url: https://www.yelp.com/biz/dennys-los-angeles-12, 93 records downloaded
restaurant url: https://www.yelp.com/biz/dona-anas-kitchen-los-angeles, 94 records downloaded
restaurant url: https://www.yelp.com/biz/dollar-hits-los-angeles-2, 95 records downloaded
restaurant url: https://www.yelp.com/biz/dominos-pizza-los-angeles-6, 96 records downloaded
restaurant url: https://www.yelp.com/biz/greenleaf-kitchen-and-cocktails-los-angeles-6, 97 records downloaded
restaurant url: https://www.yelp.com/biz/eagle-rock-green-dragon-los-angeles, 98 records downloaded
restaurant url: https://www.yelp.com/biz/ebaes-l

restaurant url: https://www.yelp.com/biz/la-villa-cafe-los-angeles-2, 176 records downloaded
restaurant url: https://www.yelp.com/biz/rays-and-stark-bar-los-angeles-2, 177 records downloaded
restaurant url: https://www.yelp.com/biz/laroccos-pizzeria-westchester-westchester, 178 records downloaded
restaurant url: https://www.yelp.com/biz/las-cazuelas-restaurant-and-pupuseria-los-angeles-3, 179 records downloaded
restaurant url: https://www.yelp.com/biz/pupuseria-las-champas-los-angeles, 180 records downloaded
restaurant url: https://www.yelp.com/biz/las-islitas-no-1-los-angeles, 181 records downloaded
restaurant url: https://www.yelp.com/biz/le-petit-paris-los-angeles-3, 182 records downloaded
restaurant url: https://www.yelp.com/biz/lenzinis-pizza-los-angeles-5, 183 records downloaded
restaurant url: https://www.yelp.com/biz/library-bar-los-angeles, 184 records downloaded
restaurant url: https://www.yelp.com/biz/library-bar-los-angeles, 185 records downloaded
restaurant url: https://ww

restaurant url: https://www.yelp.com/biz/rosalinds-ethiopian-restaurant-los-angeles, 262 records downloaded
restaurant url: https://www.yelp.com/biz/sugarfish-by-sushi-nozawa-los-angeles-10, 263 records downloaded
restaurant url: https://www.yelp.com/biz/sake-house-miro-los-angeles, 264 records downloaded
restaurant url: https://www.yelp.com/biz/se%C3%B1or-fish-los-angeles-13, 265 records downloaded
restaurant url: https://www.yelp.com/biz/shabushi-los-angeles, 266 records downloaded
restaurant url: https://www.yelp.com/biz/sonoratown-los-angeles, 267 records downloaded
restaurant url: https://www.yelp.com/biz/soregashi-los-angeles, 268 records downloaded
restaurant url: https://www.yelp.com/biz/speranza-los-angeles, 269 records downloaded
restaurant url: https://www.yelp.com/biz/spitz-eagle-rock-los-angeles-3, 270 records downloaded
restaurant url: https://www.yelp.com/biz/sqirl-los-angeles-4, 271 records downloaded
restaurant url: https://www.yelp.com/biz/sticky-rice-echo-park-los-an

In [92]:
append_df = pd.read_csv('restaurant_details_append_0409.csv')
append_df

,url,num_photos,is_bus_web,is_phone_number,is_message_bus,num_attributes,num_questions,comments_list
0,https://www.yelp.com/biz/yamakase-los-angeles,See all 2351 photos,True,False,False,19.0,See all 8 questions,"[""This is the first time I've been to the new ..."
1,https://www.yelp.com/biz/good-stuff-burgers-lo...,See all 137 photos,True,True,False,20.0,NaN,"[""Found this place on Yelp. Small and quaint e..."
2,https://www.yelp.com/biz/el-texanito-los-angeles,See all 8 photos,False,True,False,1.0,NaN,"[""Best Pupusas ever! Biggest, most flavorful, ..."
3,https://www.yelp.com/biz/aerobistro-los-angeles,See all 61 photos,True,True,False,18.0,NaN,"[""I dined lunch while I attended a seminar. Th..."
4,https://www.yelp.com/biz/pescaderia-puerto-pen...,See all 5 photos,False,True,False,10.0,NaN,"[""The Hubby and I love this place. We drive 45..."
...,...,...,...,...,...,...,...,...
329,https://www.yelp.com/biz/yamashiro-hollywood-h...,See all 8.6k photos,True,True,True,32.0,See all 189 questions,['They are famous of their view overlooking th...
330,https://www.yelp.com/biz/yamashiro-hollywood-h...,See all 8583 photos,True,True,True,32.0,See all 189 questions,['They are famous of their view overlooking th...
331,https://www.yelp.com/biz/yang-chow-los-angeles,See all 2.7k photos,True,True,False,24.0,See all 22 questions,['Customer service is out of this world the fo...
332,https://www.yelp.com/biz/yoshinoya-los-angeles-76,See all 37 photos,True,True,False,5.0,NaN,"['Nasty food, not as advertised. Pictured item..."


In [93]:
res_df = pd.concat([crawled_df,append_df],axis=1)

In [94]:
res_df.sample(10)

,FACILITY_NAME,FACILITY_ADDRESS,FACILITY_CITY,FACILITY_ID,FACILITY_STATE,FACILITY_ZIP,OWNER_ID,PE_DESCRIPTION,SCORE,VIOLATION_CODE,VIOLATION_DESCRIPTION,type,size,risk_level,ID,restaurant_id,url,name,address,score,review_counts,price,category,open_time,amenities,new_address,state,zipcode,city,url,num_photos,is_bus_web,is_phone_number,is_message_bus,num_attributes,num_questions,comments_list
329,YAMASHIRO,1999 N SYCAMORE AVE,LOS ANGELES,FA0240842,CA,90068,OW0236865,RESTAURANT (151 + ) SEATS LOW RISK,94.142857,"F006,F036,F044,F014,F033,F023,F033",# 06. Adequate handwashing facilities supplied...,restaurant,151 +,LOW,13090,219,https://www.yelp.com/biz/yamashiro-hollywood-h...,Yamashiro Hollywood,"1999 N Sycamore Ave Hollywood, CA 90068",3.5,5258 reviews,$$$,"Japanese, Asian Fusion, American (New)",Mon\n5:00 PM - 11:00 PM\nTue\n5:00 PM - 11:00 ...,Amenities and More\nHealth ScoreA\nTakes Reser...,1999 N Sycamore Ave Hollywood,CA,90068,Los Angeles,https://www.yelp.com/biz/yamashiro-hollywood-h...,See all 8.6k photos,True,True,True,32.0,See all 189 questions,['They are famous of their view overlooking th...
240,PHORAGE,3300 OVERLAND AVE STE 105,LOS ANGELES,FA0068050,CA,90034,OW0100101,RESTAURANT (31-60) SEATS HIGH RISK,92.513514,"F037,F030,F035,F033,F044,F001,F009,F026,F030,F...",# 37. Adequate ventilation and lighting; desig...,restaurant,31-60,HIGH,9089,2262,https://www.yelp.com/biz/phorage-los-angeles,Phorage,"3300 Overland Ave Los Angeles, CA 90034",3.3,(1.0k reviews),$$,Vietnamese,Mon\n11:00 AM - 10:00 PM\nTue\n11:00 AM - 10:0...,Amenities and More\nHealth ScoreA\nOffers Deli...,3300 Overland Ave Los Angeles,CA,90034,Los Angeles,https://www.yelp.com/biz/phorage-los-angeles,See all 988 photos,True,True,False,28.0,See all 4 questions,"[""I've been coming here for years. It's the pe..."
260,ROCCO'S ITALIAN MARKET&DELI,1761 N VERMONT AVE,LOS ANGELES,FA0046061,CA,90027,OW0022284,RESTAURANT (0-30) SEATS HIGH RISK,94.428571,"F035,F037,F025,F029,F025,F039,F033,F033,F035,F...",# 35. Equipment/Utensils - approved; installed...,restaurant,0-30,HIGH,9835,2702,https://www.yelp.com/biz/eastside-italian-deli...,Eastside Italian Deli - Los Feliz,"1761 N Vermont Ave Los Angeles, CA 90027",4.6,(76 reviews),$$,"Sandwiches, Italian, Delis",Mon\nClosed\nTue\n10:00 AM - 4:00 PM\nWed\n10:...,Amenities and More\nHealth ScoreA\nOffers Deli...,1761 N Vermont Ave Los Angeles,CA,90027,Los Angeles,https://www.yelp.com/biz/eastside-italian-deli...,See all 130 photos,True,True,True,23.0,See all 3 questions,"[""I didn't even know this legacy brand opened ..."
299,TAQUERIA TIJUANA,241 W FLORENCE AVE,LOS ANGELES,FA0071887,CA,90003,OW0002724,RESTAURANT (0-30) SEATS HIGH RISK,93.894737,"F031,F043,F029,F035,F036,F033,F034,F035,F029,F...",# 31. Consumer self service facilities properl...,restaurant,0-30,HIGH,11510,529,https://www.yelp.com/biz/taqueria-tijuana-los-...,Taqueria Tijuana,"241 W Florence Ave Los Angeles, CA 90003",4.5,134 reviews,$,Mexican,Mon\n6:00 AM - 8:00 PM\nTue\n6:00 AM - 8:00 PM...,Amenities and More\nHealth ScoreA\nOffers Take...,241 W Florence Ave Los Angeles,CA,90003,Los Angeles,https://www.yelp.com/biz/taqueria-tijuana-los-...,See all 104 photos,True,True,False,16.0,See all 2 questions,['10/10 would recommend.\n\nThe food was hot a...
132,HAYATO,1320 E 7TH ST # 126,LOS ANGELES,FA0252544,CA,90021,OW0246301,RESTAURANT (0-30) SEATS HIGH RISK,97.666667,"F006,F040,F033",# 06. Adequate handwashing facilities supplied...,restaurant,0-30,HIGH,4885,99,https://www.yelp.com/biz/hayato-los-angeles-3,Hayato,"1320 E 7th St Ste 126 Los Angeles, CA 90021",4.5,114 reviews,$$$$,Japanese,NaN,Amenities and More\nHealth ScoreA\nTakes Reser...,1320 E 7th St Ste 126 Los Angeles,CA,90021,Los Angeles,https://www.yelp.com/biz/hayato-los-angeles-3,See all 1.2k photos,True,True,False,19.0,See all 21 questions,"[""Hayato is no doubt the hardest reservation t..."
57,CARLS JR #212,1751 S SOTO ST,LOS ANGELES,FA0252554,CA,90023,OW0242903,RESTAURANT (61-150) SEATS MODERATE RI

In [95]:
res_df.to_csv('combined_rest_health_0409.csv', header=True, index=False)